In [15]:
pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
import pandas as pd
from IPython.display import Markdown, HTML, display

In [3]:
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
import openai

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [4]:
# Load the API key from the environment variable
api_key = os.getenv('OPENAI_API_KEY')

In [5]:
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key 
)


In [6]:
message = HumanMessage(
    content="Translate this sentence from English "
    "to French and Spanish. I like red cars and "
    "blue houses, but my dog is yellow."
)

In [7]:
model.invoke([message])

AIMessage(content="Sure! Here are the translations:\n\n**French:** J'aime les voitures rouges et les maisons bleues, mais mon chien est jaune.\n\n**Spanish:** Me gustan los coches rojos y las casas azules, pero mi perro es amarillo.")

## Load the dataset from CSV

In [8]:
os.makedirs("data", exist_ok=True)

In [9]:
# Run the wget command to download the file and save it to the "data" directory
# Run the curl command to download the file and save it to the "data" directory
!curl -o ./data/all-states-history.csv https://covidtracking.com/data/download/all-states-history.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2674k    0 2674k    0     0  5805k      0 --:--:-- --:--:-- --:--:-- 5813k


In [17]:
file_url = "./data/titanic.csv"
df = pd.read_csv("./data/titanic.csv").fillna(value = 0)

In [11]:
df

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
882,0,2,Rev. Juozas Montvila,male,27.0,0,0,13.0000
883,1,1,Miss. Margaret Edith Graham,female,19.0,0,0,30.0000
884,0,3,Miss. Catherine Helen Johnston,female,7.0,1,2,23.4500
885,1,1,Mr. Karl Howell Behr,male,26.0,0,0,30.0000


In [12]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=model,df=df,verbose=True)

agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To determine the number of rows in the dataframe `df`, I can use the `shape` attribute of the dataframe, which returns a tuple where the first element is the number of rows and the second element is the number of columns.

Action: Use the `shape` attribute to find the number of rows in the dataframe.
Action Input: df.shape
Observation: Use the `shape` attribute to find the number of rows in the dataframe. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the Python shell to execute the command and find the number of rows in the dataframe `df`.

Action: Execute the command to find the shape of the dataframe.
Action Input: df.shape
Observation: Execute the command to find the shape of the dataframe. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the correct format to execute the command in the Python shell.

Action: Execute the command to find the shape of the dataframe.
Action Input: `df.sh

{'input': 'how many rows are there?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [13]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "How many people survived" 
"of Male gender?"
"Use the Survived column" 


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: 
First, I will set the pandas display options to show all columns. Then, I will get the column names to understand the structure of the dataframe. Finally, I will count the number of people who survived by filtering the dataframe where the 'Survived' column is 1.

Action: 
The action to take is to set the pandas display options and get the column names.

Action Input: 
```python
import pandas as pd

# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Get column names
df.columns
```

Observation: The action to take is to set the pandas display options and get the column names. is not a valid tool, try one of [python_repl_ast].
Thought:I will set the pandas display options to show all columns and then get the column names to understand the structure of the dataframe. After that, I will count the number of people who survived by filtering the dataframe where the 'Survived' column is 1.

Action: 
Set the p

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow many people survived\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanation, mention the column names th

In [14]:
sum(df['Survived'])

342

In [8]:
from IPython.display import Markdown, HTML, display
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import AzureChatOpenAI

# Connect with SQLite Database

In [9]:
from sqlalchemy import create_engine


In [13]:
import sqlite3

In [14]:
# Connect to SQLite database (or create it if it doesn't exist)
database_file_path = './data/chinook.db'
connection = sqlite3.connect(database_file_path)


# Close the connection
connection.close()